# Syntax


A decorator is basically a function (or a callable class) that takes an object and returns either that object or another object.

Examples of what it might do:

- add an attribute to the function
- wrap in higher-level function
- log

Decorators can be applied to:

- functions/methods
- classes


In [1]:
def decorator_function(original_function):

    def wrapper_function(*args, **kwargs):
        print(f'Before calling {original_function.__name__}')
        result = original_function(*args, **kwargs)
        print(f'After calling {original_function.__name__}')
        return result

    return wrapper_function


@decorator_function
def greet(name):
    print(f'Hello, {name}!')


greet('Alice')

Before calling greet
Hello, Alice!
After calling greet


# Built-In Decorators

@property: This decorator allows you to define a method as a property, providing a way to access and optionally modify an attribute value. It allows you to use a method like an attribute.

@staticmethod: This decorator defines a static method within a class. Static methods are bound to the class rather than an instance, and they can be called without creating an instance of the class.

@classmethod: This decorator defines a class method within a class. Class methods receive the class itself as the first argument (usually named cls) rather than the instance. They can be called on both the class and its instances.

@abstractmethod: This decorator is used to define abstract methods within an abstract base class (ABC). Abstract methods don't have an implementation in the base class and must be implemented by the derived classes.

@classmethod: This decorator is used in conjunction with the @abstractmethod decorator to define abstract class methods within an abstract base class. Abstract class methods are bound to the class and can be called on both the class and its instances.

@functools.wraps: This decorator is used when creating decorators to preserve the original function's metadata such as name, docstring, and parameter information. It ensures that the decorated function maintains its identity.


# Decorator Resolution

Note that decorators can come from anywhere and be resolved the normal way a name is.

```Python
@mymodule.submodule.myclass.mydecorator
def f():
    pass
```

A decorator can even come from something defined in the same class (see __Properties__ for example).